In [1]:
import pandas as pd 

In [2]:
import numpy as np

In [3]:
from pymystem3 import Mystem

In [4]:
from collections import Counter

In [5]:
df = pd.read_csv('C://Users\\Igor\\Downloads\\data.csv')

## Подготовка данных

In [6]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Ищем дубликаты и удалям их

In [8]:
df = df.drop_duplicates().reset_index(drop=True)

### Удаление строк с пустыми значениями дохода

In [9]:
df[df['days_employed'].isnull() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21435,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21441,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21443,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21448,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [10]:
df.dropna(subset = ['total_income'], inplace=True)

### Приведение образования к одному формату

In [11]:
df['education'].value_counts()

среднее                12342
высшее                  4222
СРЕДНЕЕ                  705
Среднее                  646
неоконченное высшее      613
ВЫСШЕЕ                   251
Высшее                   243
начальное                231
Неоконченное высшее       40
НЕОКОНЧЕННОЕ ВЫСШЕЕ       22
НАЧАЛЬНОЕ                 16
Начальное                 14
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [12]:
df['education'] = df['education'].str.lower()

In [13]:
df['education'].value_counts()

среднее                13693
высшее                  4716
неоконченное высшее      675
начальное                261
ученая степень             6
Name: education, dtype: int64

### Замена вещественного типа данных на целочисленный

In [14]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [15]:
df['days_employed'] = df['days_employed'].astype('int64')

In [16]:
df['total_income'] = df['total_income'].astype('int64')

In [17]:
df['children'] = df['children'].astype('int64')

In [18]:
df['education_id'] = pd.to_numeric(df['education_id'], errors='raise')

### Заменяем неверное количество детей на среднее

In [19]:
df['children'].value_counts()

 0     12710
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64

In [20]:
df.loc[df['children'] < 0] = round(df['children'].mean())

In [21]:
df.loc[df['children'] > 10] = round(df['children'].mean())

### Категоризация данных

In [22]:
df.loc[df['purpose'] == 1] = np.nan

In [23]:
df.dropna(subset = ['purpose'], inplace=True)

In [24]:
estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образова']


def categoryzer(list_of_words, category):
    join = '|'.join(list_of_words)
    index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
    
    for i in index:
        df.loc[i, 'purpose_category'] = category
    return df


categoryzer(wedding, 'свадьба')
categoryzer(estate, 'недвижимость')
categoryzer(auto, 'авто')
categoryzer(education, 'образование')


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1.0,-8437.0,42.0,высшее,0.0,женат / замужем,0.0,F,сотрудник,0.0,253875.0,покупка жилья,недвижимость
1,1.0,-4024.0,36.0,среднее,1.0,женат / замужем,0.0,F,сотрудник,0.0,112080.0,приобретение автомобиля,авто
2,0.0,-5623.0,33.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,0.0,145885.0,покупка жилья,недвижимость
3,3.0,-4124.0,32.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,0.0,267628.0,дополнительное образование,образование
4,0.0,340266.0,53.0,среднее,1.0,гражданский брак,1.0,F,пенсионер,0.0,158616.0,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1.0,-4529.0,43.0,среднее,1.0,гражданский брак,1.0,F,компаньон,0.0,224791.0,операции с жильем,недвижимость
21467,0.0,343937.0,67.0,среднее,1.0,женат / замужем,0.0,F,пенсионер,0.0,155999.0,сделка с автомобилем,авто
21468,1.0,-2113.0,38.0,среднее,1.0,гражданский брак,1.0,M,сотрудник,1.0,89672.0,недвижимость,недвижимость
21469,3.0,-3112.0,38.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,1.0,244093.0,на покупку своего автомобиля,авто


## Исследование данных

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
debt_from_children = pd.DataFrame()

In [26]:
debt_from_children['count_children'] = df.groupby('children')['debt'].count()

In [27]:
debt_from_children['count_debt'] = df.groupby('children')['debt'].sum()

In [28]:
debt_from_children['result'] = debt_from_children['count_debt'] / debt_from_children['count_children']


In [29]:
debt_from_children

,count_children,count_debt,result
children,,,
0.0,12710,952.0,0.074902
1.0,4343,408.0,0.093944
2.0,1851,177.0,0.095624
3.0,294,22.0,0.074830
4.0,34,3.0,0.088235
5.0,8,0.0,0.000000


#### Вывод: у людей с 5 детьми в этих данных вообще не оказалось долгов, а у людей с двумя детьми наибольшее количество просроченных кредитов

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
debt_from_family = pd.DataFrame()

In [31]:
debt_from_family['count_family'] = df.groupby('family_status')['debt'].count()

In [32]:
debt_from_family['count_debt'] = df.groupby('family_status')['debt'].sum()

In [33]:
debt_from_family['result'] = debt_from_family['count_debt'] / debt_from_family['count_family']

In [34]:
debt_from_family

,count_family,count_debt,result
family_status,,,
Не женат / не замужем,2515,253.0,0.100596
в разводе,1078,75.0,0.069573
вдовец / вдова,858,56.0,0.065268
гражданский брак,3719,336.0,0.090347
женат / замужем,11070,842.0,0.076061


#### Вывод: меньше всего должников среди разведенных и вдовец. Больше всего должников среди не женатых.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
avg_income = df['total_income'].median()

In [36]:
debt_from_income = pd.DataFrame()

In [37]:
def income_level(total_income): 
    if total_income < avg_income:
        return 'меньше'
    return 'больше'

In [38]:
df['income_level'] = df['total_income'].apply(income_level)

In [39]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_level
0,1.0,-8437.0,42.0,высшее,0.0,женат / замужем,0.0,F,сотрудник,0.0,253875.0,покупка жилья,недвижимость,больше
1,1.0,-4024.0,36.0,среднее,1.0,женат / замужем,0.0,F,сотрудник,0.0,112080.0,приобретение автомобиля,авто,меньше
2,0.0,-5623.0,33.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,0.0,145885.0,покупка жилья,недвижимость,больше
3,3.0,-4124.0,32.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,0.0,267628.0,дополнительное образование,образование,больше
4,0.0,340266.0,53.0,среднее,1.0,гражданский брак,1.0,F,пенсионер,0.0,158616.0,сыграть свадьбу,свадьба,больше
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1.0,-4529.0,43.0,среднее,1.0,гражданский брак,1.0,F,компаньон,0.0,224791.0,операции с жильем,недвижимость,больше
21467,0.0,343937.0,67.0,среднее,1.0,женат / замужем,0.0,F,пенсионер,0.0,155999.0,сделка с автомобилем,авто,больше
21468,1.0,-2113.0,38.0,среднее,1.0,гражданский брак,1.0,M,сотрудник,1.0,89672.0,недвижимость,недвижимость,меньше
21469,3.0,-3112.0,38.0,среднее,1.0,женат / замужем,0.0,M,сотрудник,1.0,244093.0,на покупку своего автомобиля,авто,больше


In [40]:
debt_from_income['count_level'] = df.groupby('income_level')['debt'].count()

In [41]:
debt_from_income['sum_debt'] = df.groupby('income_level')['debt'].sum()

In [42]:
debt_from_income['result'] = debt_from_income['sum_debt'] / debt_from_income['count_level'] 

In [43]:
debt_from_income

,count_level,sum_debt,result
income_level,,,
больше,9620,762.0,0.07921
меньше,9620,800.0,0.08316


#### Вывод: у людей с большим (больше среднего) доходом меньше просроченных кредитов, чем у людей с мальеникм (меньше среднего)

### Как разные цели кредита влияют на его возврат в срок?

In [44]:
debt_from_purpose = pd.DataFrame()

In [45]:
debt_from_purpose['count_purpose'] = df.groupby('purpose_category')['debt'].count()

In [46]:
debt_from_purpose['count_debt'] = df.groupby('purpose_category')['debt'].sum()

In [47]:
debt_from_purpose['result'] = debt_from_purpose['count_debt'] / debt_from_purpose['count_purpose']


In [48]:
debt_from_purpose

,count_purpose,count_debt,result
purpose_category,,,
авто,3872,364.0,0.094008
недвижимость,9704,713.0,0.073475
образование,3575,330.0,0.092308
свадьба,2089,155.0,0.074198


#### Вывод: чаще всего возвращают в срок кредит на недвижимость. Чаще всего просрачивают кредит на авто. Но разница между всеми категориями очень мала